In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from datetime import datetime
import matplotlib.dates as mdates
import scipy
import os
from scipy.stats import linregress
from datetime import timedelta

import matplotlib.dates as mdates
import matplotlib.gridspec as gridspec
import matplotlib.ticker as ticker

plt.rcParams['figure.dpi'] = 300

## 1. Load in LAI data

In [72]:
lai_files_dir = 'C:/Users/jeanallen/Desktop/SHIFT_CWC/data/field_data'

# load in files
lai_1_path = os.path.join(lai_files_dir, 'LAI_20220427_20220428.xls')
lai_2_path = os.path.join(lai_files_dir, 'LAI_20220518_20220521.xls')
lai_3_path = os.path.join(lai_files_dir, 'LAI_20220526_20220527.xls')
lai_4_path = os.path.join(lai_files_dir, 'lai_20220915.xls')
lai_1 = pd.read_excel(lai_1_path).dropna()
lai_2 = pd.read_excel(lai_2_path).dropna()
lai_3 = pd.read_excel(lai_3_path).dropna()
lai_4 = pd.read_excel(lai_4_path).dropna()

# isolate just the "Round" measurements, which are an average from 15-20 places under the tree canopy
lai_1 = lai_1[lai_1['Direction'] == 'Round']
lai_2 = lai_2[lai_2['Direction'] == 'Round']
lai_3 = lai_3[lai_3['Direction'] == 'Round']
lai_4 = lai_4[lai_4['Direction'] == 'Round']

# switch the columns "Average Above PAR" and "Average Below PAR"
lai_1['Average Above PAR'], lai_1['Average Below PAR'] = lai_1['Average Below PAR'], lai_1['Average Above PAR']
lai_2['Average Above PAR'], lai_2['Average Below PAR'] = lai_2['Average Below PAR'], lai_2['Average Above PAR']
lai_3['Average Above PAR'], lai_3['Average Below PAR'] = lai_3['Average Below PAR'], lai_3['Average Above PAR']
lai_4['Average Above PAR'], lai_4['Average Below PAR'] = lai_4['Average Below PAR'], lai_4['Average Above PAR']

# get rid of the "Direction" column
lai_1 = lai_1.drop(columns=['Direction'])
lai_2 = lai_2.drop(columns=['Direction'])
lai_3 = lai_3.drop(columns=['Direction'])
lai_4 = lai_4.drop(columns=['Direction'])

## 2. Join with PAR data

In [73]:
# load in par data
par_data_path = 'C:/Users/jeanallen/Desktop/SHIFT_CWC/data/RidgeTop_complete.xlsx'
par_data = pd.read_excel(par_data_path)
par_data

,RidgeTop,PAR umol/m²s,RH,TEMP (C),Pressure (kPa),Precip (mm),Port 4,Port 5
0,2022-03-24 18:05:00,261.840820,0.478406,20.600000,77.798241,0.0,NaN,NaN
1,2022-03-24 18:10:00,373.535156,0.436297,25.700001,97.260872,0.0,NaN,NaN
2,2022-03-24 18:15:00,336.914062,0.435374,25.500000,97.260872,0.0,NaN,NaN
3,2022-03-24 18:20:00,294.799805,0.442206,25.000000,97.260872,0.0,NaN,NaN
4,2022-03-24 18:25:00,258.178711,0.443190,24.600000,97.260872,0.0,NaN,NaN
...,...,...,...,...,...,...,...,...
56117,2022-10-05 14:30:00,1426.391602,0.297982,34.500000,97.099655,0.0,NaN,NaN
56118,2022-10-05 14:35:00,1404.418945,0.303956,34.900002,97.114311,0.0,NaN,NaN
56119,2022-10-05 14:40:00,1384.277344,0.316472,34.900002,97.114311,0.0,NaN,NaN
56120,2022-10-05 14:45:00,1362.304688,0.302050,34.799999,97.099655,0.0,NaN,NaN


In [74]:
# function to take a time and go get the coincident PAR data
def getPAR(date_and_time):
    # find PAR data that was within five minutes
    par_data_relevant = par_data[(par_data['RidgeTop'] >= date_and_time - timedelta(minutes=5)) & (par_data['RidgeTop'] <= date_and_time + timedelta(minutes=5))]
    # if there's just one value, return it
    if len(par_data_relevant) == 1:
        return par_data_relevant['PAR umol/m²s'].iloc[0]
    # if there are multiple values, interpolate across them
    elif len(par_data_relevant) > 1:
        return np.interp(date_and_time.timestamp(), par_data_relevant['RidgeTop'].apply(lambda x: x.timestamp()), par_data_relevant['PAR umol/m²s'])

In [75]:
# apply this function and store it as "Average Above PAR"
lai_1['Average Above PAR'] = lai_1['Date and Time'].apply(getPAR)
lai_2['Average Above PAR'] = lai_2['Date and Time'].apply(getPAR)
lai_3['Average Above PAR'] = lai_3['Date and Time'].apply(getPAR)
lai_4['Average Above PAR'] = lai_4['Date and Time'].apply(getPAR)

In [76]:
# # save out the modified dataframes
# lai_1.to_csv(os.path.join(lai_files_dir, 'lai_1_round.csv'), index=False)
# lai_2.to_csv(os.path.join(lai_files_dir, 'lai_2_round.csv'), index=False)
# lai_3.to_csv(os.path.join(lai_files_dir, 'lai_3_round.csv'), index=False)
# lai_4.to_csv(os.path.join(lai_files_dir, 'lai_4_round.csv'), index=False)

# the rest of the LAI math will be done in the spreadsheets provided by METER